In [1]:
# 库文件
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from skopt import gp_minimize
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import train_test_split

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics.pairwise import rbf_kernel
import optuna

import numpy as np
from sklearn.svm import SVR
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# 从 Excel 文件读取数据
df = pd.read_excel('data.xlsx')

# 提取列数据
u1_data = df['u1'].values
u2_data = df['u2'].values
y1_data = df['y1'].values
y2_data = df['y2'].values

# # 输出提取的数据
# print('u1_data:', u1_data.shape)
# print('u2_data:', u2_data.shape)
# print('y1_data:', y1_data.shape)
# print('y2_data:', y1_data.shape)

num_samples = y1_data.shape[0]


In [3]:
# 组合训练数据  多输入单输出
# 假设有训练数据 u1_data, u2_data, y1_data 和 y1_data+1
# 将它们组合成输入特征矩阵 X 和输出标签向量 y
u1_data = u1_data
u2_data = u2_data

u1_data_1 = np.roll(u1_data, 1)
u2_data_1 = np.roll(u2_data, 1)
u1_data_1[0], u2_data_1[0] = 0, 0

y1_data = y1_data

X = np.column_stack((u1_data[:-1], u2_data[:-1], u1_data_1[:-1], u2_data_1[:-1], y1_data[:-1]))
y = y1_data[1:]

print(X.shape,y.shape)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33333, random_state=42)


(300, 5) (300,)


In [4]:
# 定义My_LS_SVRModel
class My_LS_SVRModel:
    def __init__(self, params):
        self.C = params[0]
        self.gamma = params[1]
        self.W_b = None
        self.X_train = None

    def model_train(self, X_train, y_train, K_train):
        def objective(W_b, X, y, C):
            y_pred = np.dot(K_train, W_b[:-1]) + W_b[-1]
            errors = y - y_pred
            loss = 0.5 * np.dot(W_b[:-1], W_b[:-1]) + C * np.sum(errors**2)
            return loss

        initial_W_b = np.zeros(X_train.shape[0] + 1)
        result = minimize(objective, initial_W_b, args=(X_train, y_train, self.C), method='L-BFGS-B')
        best_W_b = result.x
        return best_W_b

    def fit(self, X_train, y_train):
        self.X_train = X_train
        K_train = rbf_kernel(X_train, X_train, gamma=self.gamma)
        self.W_b = self.model_train(X_train, y_train, K_train)

    def predict(self, X_test):
        K_test = rbf_kernel(X_test, self.X_train, gamma=self.gamma)
        y_pred = np.dot(K_test, self.W_b[:-1]) + self.W_b[-1]
        return y_pred
    

# # 示例用法
# params = [8.78525340e+01, 2.01347249e-03]
# # 创建模型
# my_svr_model = My_LS_SVRModel(params=params)
# # 训练模型
# my_svr_model.fit(X_train, y_train)
# # 模型预测
# y_pred = my_svr_model.predict(X_test)


In [29]:

def your_fitness_function(trial, params, X_train, y_train, X_test, y_test):
    # 示例用法
    # 创建模型
    my_svr_model = My_LS_SVRModel(params=params)
    # 训练模型
    my_svr_model.fit(X_train, y_train)
    # 模型预测
    y_pred = my_svr_model.predict(X_test)        
    # 在这里计算适应值（均方根误差）
    errors = y_test - y_pred
    mse = np.sqrt(np.sum(errors**2)/X_test.shape[0])

    return mse


# 目标函数适配 Optuna 风格
def objective(trial, X_train, y_train, X_test, y_test):
    # 在搜索空间中定义超参数
    C = trial.suggest_float('C', 0.1, 20.0, log=True)
    gamma = trial.suggest_float('gamma', 1e-9, 1e-2, log=True)

    # 调用你的目标函数
    params = [C, gamma]
    mse = your_fitness_function(trial, params, X_train, y_train, X_test, y_test)
    
    # 返回目标值（Optuna 默认是最小化目标）
    return mse

# 创建 Optuna 试验对象
# 创建 Optuna 试验对象，指定采样器为粒子群算法
sampler = optuna.samplers.PSO()
study = optuna.create_study(direction='minimize', sampler=sampler)

# 执行优化
study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=100)

# 输出结果
print(f"Best Trial: {study.best_trial.params}")
print(f"Best MSE: {study.best_value}")




AttributeError: module 'optuna.samplers' has no attribute 'PSO'

In [28]:
print("Sampler: ", study.sampler)
print("Pruner: ", study.pruner)


Sampler:  TPESampler
Pruner:  <optuna.pruners._median.MedianPruner object at 0x0000018EBC1E5AF0>


In [26]:
optuna.visualization.plot_contour(study)


In [27]:
# 绘制优化过程
optuna.visualization.plot_optimization_history(study)


In [ ]:
# 创建模型
#  data    [5.33702937e+01 2.46106482e-02]  0.29159782217670577
#  data2   [3.04694015e+01 2.11002630e-02]  0.3105070871640635
params =   [70.21959218287664 ,0.026466571352123314]
my_svr_model = My_LS_SVRModel(params=params)
# 训练模型
my_svr_model.fit(X_train, y_train)
# 模型预测
y_pred = my_svr_model.predict(X_test)


In [ ]:
# 计算 RMSE
y_test = y_test
rmse_0 = np.sqrt(mean_squared_error(y_test, y_pred))

# 计算 MRE
mre_0 = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# 打印结果
print("Dimension 0:")
print("RMSE:", rmse_0)
print("MRE:", mre_0)



In [ ]:
# 模型预测结果
plt.figure(figsize=(12, 6)) 
plt.plot(y_test,'b-')
plt.plot(y_pred,'r--')


In [ ]:
# 生成参考轨迹
def get_yr(aim_value,current_value,alpha,P):

    # 生成设定信号
    setpoint_value = aim_value
    setpoint_signal = np.full(10, setpoint_value)
    # 初始化参数
    alpha = alpha
    y_r = np.zeros(P)
    y_r[0] = current_value
    # 模拟一阶模型
    for k in range(1,P):
        y_r[k] = alpha * y_r[k-1] + (1 - alpha) * setpoint_value


    # # 绘制结果
    # plt.plot(setpoint_signal, label='Setpoint Signal')
    # plt.plot(y_r,'o-', label='Output Signal (Tracked)')
    # plt.legend()
    # plt.xlabel('Time')
    # plt.ylabel('Amplitude')
    # plt.title('Tracking Setpoint Signal with One-Order Model')
    # plt.show()

    # print('y_r000',y_r)
    return y_r
# 测试
y_r = get_yr(-1,-0.5,0.333,5)


In [ ]:
# 生成期望数据
Times = 200

def generate_y_aim_data(Times):
    set_y1 = np.full(Times, 0.5)
    set_y1[30:51] = 0.6
    set_y1[80:101] = 0.7
    set_y1[130:151] = 0.0
    set_y1[170:201] = 0.4
    set_y1[185] = -0.7  # 添加脉冲干扰

    set_y2 = np.full(Times, 0.3)
    set_y2[50:81] = -0.1
    set_y2[100:131] = 0.5
    set_y2[150:171] = -0.7  # 添加脉冲干扰
    set_y2[200:] = 0.4

    # 限制设定值在 -1 到 1 之间
    set_y1 = np.clip(set_y1, -1, 1)
    set_y2 = np.clip(set_y2, -1, 1)

    return set_y1, set_y2



# 调用示例
set_y1, set_y2 = generate_y_aim_data(Times)
plt.plot(set_y1)
plt.plot(set_y2)


In [ ]:
#生成控制时域的数据格式
def generate_k_data(u1_data, u2_data, y1_data, num_samples, P):
    # 生成随机索引值
    #从原有数据的randint时刻开始往下进行控制
    randint = np.random.randint(1, num_samples - 2 - P - 1)
    
    # randint = 250  # 如果你希望使用固定的值而不是随机生成
    print(randint)
    # 提取数据并构成 k_data
    # 第一次得到下面五个变量，固定好格式构成k_data
    u1   = u1_data[randint  :randint+2  ]
    u2   = u2_data[randint  :randint+2  ]
    u1_1 = u1_data[randint-1:randint+2-1]
    u2_1 = u2_data[randint-1:randint+2-1]
    y1   = y1_data[randint  :randint+2  ]
    k_data = np.concatenate((u1, u2, u1_1, u2_1, y1), axis=0)
    # print(k_data.shape)
    return k_data


In [ ]:
# 定义单时刻的MPC问题优化
def my_MPC(k_data,params,M,P,y1_aim):
    # 从固定格式k_data里面读取信息
    u1   = k_data[0:2]
    u2   = k_data[2:4]
    u1_1 = k_data[4:6]
    u2_1 = k_data[6:8]
    y1   = k_data[8:10]
    # 获取猜测值[h U1 U2]
    h, U1, U2  =params[0], params[1:M+1],params[M+1:]
    
    # 整理数据见   MPC推到.escel
    u1   = np.concatenate((u1[:1],U1))
    u2   = np.concatenate((u2[:1],U2))
    u1_1 = np.concatenate((u1_1[:2],U1[:-1]))
    u2_1 = np.concatenate((u2_1[:2],U2[:-1]))
    y1   = np.concatenate((y1,np.zeros(P)))
    y1_k = y1[1]
    # print(u1)
    # print(u2)
    # print(u1_1)
    # print(u2_1)
    # print(y1)
    # print(y1_k)
    
    # print('1111111111111111111111111')

    # 总共预测 P+1 次
    # 对k时刻进行预测-----1次
    for j in range(1):
        x = np.column_stack((u1[j],u2[j],u1_1[j],u2_1[j],y1[j]))
        y1_m_k = my_svr_model.predict(x)
        # y1[j+1] = y1_m_k.item()###############################是否替换不知道有没有影响？==不该替换
        # 在k时刻，我要先通过k-1时刻来预测出当前k时刻下的预测值。
        # 实际上，这个时候我有k时刻的真实值。
        # 但是这么做是为了能够获取我的预测值和真实值之间的误差，
        # 然后基于这个误差，通过对控制器输入的调整来预测k+1时刻，
        # 然后依次往下。因此，在k时刻，我系统的值是固定的，
        # 也就是我的真实值，所以我要把它存储到我的整体的一个序列里面。
        # 在取出当前k时刻真实值的过程中，我不能将之前预测的时候的那个
        # 预测值覆盖掉真实值。后面新预测的数值要加上前面的那一个误差
        E_k = y1_k - y1_m_k

    # 对每个U对应的控制时刻进行预测-----M次
    for j in range(1,M+1):     
        x = np.column_stack((u1[j],u2[j],u1_1[j],u2_1[j],y1[j]))
        y1_k_j = my_svr_model.predict(x)
        y1[j+1] = y1_k_j.item()#将预测值作为下一步的输出值

    # 对控制时域外的部分进行预测-----P-M次
    # 注意：这部分的信号是保持控制不变下进行
    for j in range(M+1,P+1):
        x = np.column_stack((u1[-1],u2[-1],u1[-1],u2[-1],y1[j]))
        y1_k_j = my_svr_model.predict(x)
        y1[j+1] = y1_k_j.item()

    # print(u1)
    # print(u2)
    # print(u1_1)
    # print(u2_1)
    # print(y1)
    # print(y1_k)
    # print('2222222222222222222')
    
    #和获取参考轨迹
    y_r = get_yr(y1_aim,y1_k,0.1,P+1)
    y1_r = y_r[1:]
    y1_M_k = y1[2:]
    # print('y1_aim',y1_aim)
    # print('y_r',y_r)
    # print('y1',y1)
    # print('h*E_k',h*E_k)

    # 计算mse
    # lamda1太大的话会导致y1_r和y1_M_k的误差加大*****************导致超调的原因
    lamda1 = 0.1
    lamda2 = 0.1
    mse = (np.sum((y1_r - y1_M_k - h*E_k)**2)
            +np.sum(lamda1*(np.diff(u1)**2))
            +np.sum(lamda2*(np.diff(u2)**2))
            )
    # print('mse',mse)
    
    # 生成新的k_data2
    k_data2 = np.concatenate((u1[1:3],u2[1:3],u1_1[1:3],u2_1[1:3],y1[1:3]),axis=0)

    return mse , y1[1], U1[0], U2[0] , k_data2




In [ ]:
# 对未来Times周期预测控制

# 期望设定值
set_y1 , set_y2 = generate_y_aim_data(Times)
# MPC参数
P = 3  # 预测时域长度
M = 2  # 控制时域长度
#生成控制时域的数据格式
k_data = generate_k_data(u1_data, u2_data, y1_data, num_samples, P)

# MPC控制循环   迭代的只有：k_data
all_pred_y1 = []
all_pred_y2 = []
all_pred_u1 = []
all_pred_u2 = []
for k in range(Times):
    print(f"这是对第{k}时刻的最优U1、U2输入求解")

    # 定义优化目标函数
    def objective_function(params, *k_data):
        mse, y1m_k, u1, u2, k_data2 = my_MPC(k_data = k_data[0], params=params, M=M, P=P, y1_aim = set_y1[k])     
        return mse
    
    # 初始猜测值[h U1 U2]
    params = np.concatenate([np.array([0.1]), np.ones(M), np.ones(M)])
    # 定义参数的上下限  
    bounds = [(0, 1)] + [(-1, 1) for _ in range(2 * M)]
    # 进行优化  
    result = minimize(objective_function, params, method='L-BFGS-B', bounds = bounds, args = k_data)#args传进来的是一个元组
    optimized_h, optimized_U1, optimized_U2 = result.x[0], result.x[1:M+1], result.x[M+1:]
    # print('333333333333333333')
    # 获取当前时刻下，在最优的U1、U2下的响应
    params = np.concatenate(([optimized_h],optimized_U1,optimized_U2),axis=0)
    mse , y1m_k, u1, u2, k_data2=my_MPC(k_data=k_data,params=params,M=M,P=P, y1_aim = set_y1[k])
    all_pred_y1.append(y1m_k)
    all_pred_u1.append(u1)
    all_pred_u2.append(u2)
    k_data = k_data2
    # print('4444444444444444') 
    # 进入下一时刻，更新预测时域、控制时域，即k_data


print(all_pred_y1)


In [ ]:
# 模型预测控制结果可视化
# 创建两个子图，分别绘制每个维度
plt.figure(figsize=(12, 12))

# 第一个维度的曲线
plt.subplot(4, 1, 1)
plt.plot(set_y1, 'ro-', label='set_y1')
plt.plot(all_pred_y1, 'bo-', label='all_pred_y1')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Comparison of set_y1 and all_pred_y1')
plt.legend()

# 第二个维度的曲线
plt.subplot(4, 1, 2)
plt.plot(set_y2, 'ro-', label='set_y2')
plt.plot(all_pred_y2, 'bo-', label='all_pred_y2')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Comparison of set_y2 and all_pred_y2')
plt.legend()

# 第一个维度的u1曲线
plt.subplot(4, 1, 3)
plt.plot(all_pred_u1, 'bo-', label='all_pred_u1')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Prediction of all_pred_u1')
plt.legend()

# 第二个维度的u2曲线
plt.subplot(4, 1, 4)
plt.plot(all_pred_u2, 'bo-', label='all_pred_u2')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Prediction of all_pred_u2')
plt.legend()

# 调整子图布局
plt.tight_layout()
plt.show()
